## Lab 2
October 8, 2025
Sarah Innis

This notebook is designed to be run within Conda 516lab2 environment.

## Import Libraries

In [56]:
import polars as pl

## Initial Data Inspection
Prior to using polars, the command line was used to inspect files.

Here are the results to the commands to view file size: 
$ ls -lh data/*.csv
-rw-r--r-- 1 sarah 197609 3.4K Sep 21 18:18 data/datacenter_locations.csv
-rw-r--r-- 1 sarah 197609 174K Sep 21 18:18 data/order_detail_2023_Q1.csv
-rw-r--r-- 1 sarah 197609 175K Sep 21 18:18 data/order_detail_2023_Q2.csv
-rw-r--r-- 1 sarah 197609 175K Sep 21 18:18 data/order_detail_2023_Q3.csv
-rw-r--r-- 1 sarah 197609 175K Sep 21 18:18 data/order_detail_2023_Q4.csv
-rw-r--r-- 1 sarah 197609 214K Sep 21 18:18 data/order_summary_2023.csv

These files are not super big that I would expect any of them to break my computer when I open them.

From previewing the first few lines I can see the data is comma delimited and the files have the following columns:
- datacenter_locations.csv: dc_code,country,city,location_name,total_power_mw,available_power_mw,tier_level,commissioning_date
- order_detail files for each quarter in 2023: order_number,line_item_id,item_category,item_description,quantity,unit_price,item_total
- order_summary_2023.csv: order_number,order_date,total_amount,vendor,destination_dc_code

Counted the total rows of each file using wc -l filename.csv
- datacenter_locations.csv: 51
- order_detail_2023_Q1.csv: 2820
- order_detail_2023_Q2.csv: 2820
- order_detail_2023_Q3.csv: 2820
- order_detail_2023_Q4.csv: 2820
- order_summary_2023.csv: 3501

I ran grep -n ',,' filename.csv to look for missing values and did not find any.

## Data Loading and Inspection

In [57]:
datacenter = pl.read_csv("./data/datacenter_locations.csv")
datacenter.head()

dc_code,country,city,location_name,total_power_mw,available_power_mw,tier_level,commissioning_date
str,str,str,str,f64,f64,i64,str
"""DC-UK001""","""United Kingdom""","""London""","""London DC-1""",88.59,22.94,3,"""2020-01-11"""
"""DC-US001""","""United States""","""San Jose""","""San Jose DC-1""",11.92,4.21,3,"""2012-01-16"""
"""DC-JP001""","""Japan""","""Osaka""","""Osaka DC-1""",15.4,5.83,2,"""2015-04-17"""
"""DC-IE001""","""Ireland""","""Dublin""","""Dublin DC-1""",14.2,2.22,3,"""2014-08-14"""
"""DC-DE001""","""Germany""","""Frankfurt""","""Frankfurt DC-1""",30.0,1.99,4,"""2021-04-16"""


Inspect the schema and number of rows. There are 50 rows in the datacenter dataframe. Data types look correct except for commissioning_date.

In [58]:
print(datacenter.schema)
print(datacenter.height)

Schema({'dc_code': String, 'country': String, 'city': String, 'location_name': String, 'total_power_mw': Float64, 'available_power_mw': Float64, 'tier_level': Int64, 'commissioning_date': String})
50


In [59]:
# Count nulls per column
null_counts = datacenter.select([pl.col(col).is_null().sum().alias(col) for col in datacenter.columns])
print(null_counts)

shape: (1, 8)
┌─────────┬─────────┬──────┬──────────────┬──────────────┬──────────────┬────────────┬─────────────┐
│ dc_code ┆ country ┆ city ┆ location_nam ┆ total_power_ ┆ available_po ┆ tier_level ┆ commissioni │
│ ---     ┆ ---     ┆ ---  ┆ e            ┆ mw           ┆ wer_mw       ┆ ---        ┆ ng_date     │
│ u32     ┆ u32     ┆ u32  ┆ ---          ┆ ---          ┆ ---          ┆ u32        ┆ ---         │
│         ┆         ┆      ┆ u32          ┆ u32          ┆ u32          ┆            ┆ u32         │
╞═════════╪═════════╪══════╪══════════════╪══════════════╪══════════════╪════════════╪═════════════╡
│ 0       ┆ 0       ┆ 0    ┆ 0            ┆ 0            ┆ 0            ┆ 0          ┆ 0           │
└─────────┴─────────┴──────┴──────────────┴──────────────┴──────────────┴────────────┴─────────────┘


In [60]:
#make commissioning_date date format
datacenter_format = datacenter.with_columns([
    pl.col("commissioning_date").str.strptime(pl.Date).alias("commissioning_date")
])
print(datacenter_format.schema)

Schema({'dc_code': String, 'country': String, 'city': String, 'location_name': String, 'total_power_mw': Float64, 'available_power_mw': Float64, 'tier_level': Int64, 'commissioning_date': Date})


In [61]:
order_detail_q1 = pl.read_csv("./data/order_detail_2023_Q1.csv")
order_detail_q1.head()

# Count nulls per column
null_counts = order_detail_q1.select([pl.col(col).is_null().sum().alias(col) for col in order_detail_q1.columns])
print(null_counts)

shape: (1, 7)
┌──────────────┬──────────────┬───────────────┬───────────────┬──────────┬────────────┬────────────┐
│ order_number ┆ line_item_id ┆ item_category ┆ item_descript ┆ quantity ┆ unit_price ┆ item_total │
│ ---          ┆ ---          ┆ ---           ┆ ion           ┆ ---      ┆ ---        ┆ ---        │
│ u32          ┆ u32          ┆ u32           ┆ ---           ┆ u32      ┆ u32        ┆ u32        │
│              ┆              ┆               ┆ u32           ┆          ┆            ┆            │
╞══════════════╪══════════════╪═══════════════╪═══════════════╪══════════╪════════════╪════════════╡
│ 0            ┆ 0            ┆ 0             ┆ 0             ┆ 0        ┆ 0          ┆ 0          │
└──────────────┴──────────────┴───────────────┴───────────────┴──────────┴────────────┴────────────┘


In [62]:
order_detail_q2 = pl.read_csv("./data/order_detail_2023_Q2.csv")
order_detail_q2.head()

# Count nulls per column
null_counts = order_detail_q2.select([pl.col(col).is_null().sum().alias(col) for col in order_detail_q2.columns])
print(null_counts)

shape: (1, 7)
┌──────────────┬──────────────┬───────────────┬───────────────┬──────────┬────────────┬────────────┐
│ order_number ┆ line_item_id ┆ item_category ┆ item_descript ┆ quantity ┆ unit_price ┆ item_total │
│ ---          ┆ ---          ┆ ---           ┆ ion           ┆ ---      ┆ ---        ┆ ---        │
│ u32          ┆ u32          ┆ u32           ┆ ---           ┆ u32      ┆ u32        ┆ u32        │
│              ┆              ┆               ┆ u32           ┆          ┆            ┆            │
╞══════════════╪══════════════╪═══════════════╪═══════════════╪══════════╪════════════╪════════════╡
│ 0            ┆ 0            ┆ 0             ┆ 0             ┆ 0        ┆ 0          ┆ 0          │
└──────────────┴──────────────┴───────────────┴───────────────┴──────────┴────────────┴────────────┘


In [63]:
order_detail_q3 = pl.read_csv("./data/order_detail_2023_Q3.csv")
order_detail_q3.head()

# Count nulls per column
null_counts = order_detail_q3.select([pl.col(col).is_null().sum().alias(col) for col in order_detail_q3.columns])
print(null_counts)

shape: (1, 7)
┌──────────────┬──────────────┬───────────────┬───────────────┬──────────┬────────────┬────────────┐
│ order_number ┆ line_item_id ┆ item_category ┆ item_descript ┆ quantity ┆ unit_price ┆ item_total │
│ ---          ┆ ---          ┆ ---           ┆ ion           ┆ ---      ┆ ---        ┆ ---        │
│ u32          ┆ u32          ┆ u32           ┆ ---           ┆ u32      ┆ u32        ┆ u32        │
│              ┆              ┆               ┆ u32           ┆          ┆            ┆            │
╞══════════════╪══════════════╪═══════════════╪═══════════════╪══════════╪════════════╪════════════╡
│ 0            ┆ 0            ┆ 0             ┆ 0             ┆ 0        ┆ 0          ┆ 0          │
└──────────────┴──────────────┴───────────────┴───────────────┴──────────┴────────────┴────────────┘


In [64]:
order_detail_q4 = pl.read_csv("./data/order_detail_2023_Q4.csv")
order_detail_q4.head()

# Count nulls per column
null_counts = order_detail_q4.select([pl.col(col).is_null().sum().alias(col) for col in order_detail_q4.columns])
print(null_counts)

shape: (1, 7)
┌──────────────┬──────────────┬───────────────┬───────────────┬──────────┬────────────┬────────────┐
│ order_number ┆ line_item_id ┆ item_category ┆ item_descript ┆ quantity ┆ unit_price ┆ item_total │
│ ---          ┆ ---          ┆ ---           ┆ ion           ┆ ---      ┆ ---        ┆ ---        │
│ u32          ┆ u32          ┆ u32           ┆ ---           ┆ u32      ┆ u32        ┆ u32        │
│              ┆              ┆               ┆ u32           ┆          ┆            ┆            │
╞══════════════╪══════════════╪═══════════════╪═══════════════╪══════════╪════════════╪════════════╡
│ 0            ┆ 0            ┆ 0             ┆ 0             ┆ 0        ┆ 0          ┆ 0          │
└──────────────┴──────────────┴───────────────┴───────────────┴──────────┴────────────┴────────────┘


In [65]:
order_summary = pl.read_csv("./data/order_summary_2023.csv")
order_summary.head()

order_number,order_date,total_amount,vendor,destination_dc_code
str,str,f64,str,str
"""PO-2023-000001""","""2023-01-16""",287622.53,"""Dell Technologies""","""DC-JP001"""
"""PO-2023-000002""","""2023-12-24""",60224.27,"""Juniper Networks""","""DC-DE001"""
"""PO-2023-000003""","""2023-10-22""",474361.49,"""Super Micro Computer""","""DC-US013"""
"""PO-2023-000004""","""2023-09-20""",697759.98,"""HPE (Hewlett Packard Enterpris…","""DC-NL004"""
"""PO-2023-000005""","""2023-06-22""",83640.85,"""Lenovo""","""DC-BR001"""


Inspect the schema and number of rows. There are 3500 rows in the datacenter dataframe. Data types look correct except for order_date.

In [66]:
print(order_summary.schema)
print(order_summary.height)

Schema({'order_number': String, 'order_date': String, 'total_amount': Float64, 'vendor': String, 'destination_dc_code': String})
3500


In [67]:
# Count nulls per column
null_counts = order_summary.select([pl.col(col).is_null().sum().alias(col) for col in order_summary.columns])
print(null_counts)

shape: (1, 5)
┌──────────────┬────────────┬──────────────┬────────┬─────────────────────┐
│ order_number ┆ order_date ┆ total_amount ┆ vendor ┆ destination_dc_code │
│ ---          ┆ ---        ┆ ---          ┆ ---    ┆ ---                 │
│ u32          ┆ u32        ┆ u32          ┆ u32    ┆ u32                 │
╞══════════════╪════════════╪══════════════╪════════╪═════════════════════╡
│ 0            ┆ 0          ┆ 0            ┆ 0      ┆ 0                   │
└──────────────┴────────────┴──────────────┴────────┴─────────────────────┘


In [68]:
#make order_date date format
order_summary_format = order_summary.with_columns([
    pl.col("order_date").str.strptime(pl.Date).alias("order_date")
])
print(order_summary_format.schema)

Schema({'order_number': String, 'order_date': Date, 'total_amount': Float64, 'vendor': String, 'destination_dc_code': String})


Select only the columns you need for the analysis (e.g., order_number, vendor, total_amount, destination_dc_code from summary; dc_code, country from locations)

In [69]:
order_summary_subset = order_summary_format.select(['order_number', 'vendor', 'total_amount', 'destination_dc_code'])
datacenter_subset = datacenter_format.select(['dc_code', 'country'])

### Join, Aggregate, and Validate

Goals:
- Vendor spend by country (2023)
- Validate order totals using 2023 detail lines and identify mismatches

In [70]:
df = (
    # inner join orders and locations on the dc code
    order_summary_subset.join(datacenter_subset, left_on = 'destination_dc_code', right_on = 'dc_code', how = 'inner')
    # group by country and vendor_code
    .group_by("country", "vendor")
    # aggregate and calculate total spend and order_count
    .agg([
        pl.col("total_amount").sum().alias("total_spend"),
        pl.count("order_number").alias("order_count")
    ])
    # for each country, keep the top 5 vendors by total_spend and sort by country, then total_spend desc
    .with_columns(
        pl.col("total_spend")
        .rank(method = "dense", descending = True)
        .over("country")
        .alias("rank_within_country")
    )
    # keep top 5 vendors per country
    .filter(pl.col("rank_within_country") <= 5)
    # sort by country then total_spend descending
    .sort(["country", "total_spend"], descending = [False, True])
    # drop the rank column
    .drop("rank_within_country")
)
df.head()

country,vendor,total_spend,order_count
str,str,f64,u32
"""Australia""","""Dell Technologies""",1.5225e7,31
"""Australia""","""Cisco Systems""",1.4512e7,30
"""Australia""","""Lenovo""",1.0356e7,21
"""Australia""","""Super Micro Computer""",4489828.4,11
"""Australia""","""HPE (Hewlett Packard Enterpris…",4.1630e6,12


In [71]:
detail_df = (
    # concatenate dataframes with order details from each quarter of 2023
    pl.concat([order_detail_q1, order_detail_q2, order_detail_q3, order_detail_q4])
    # group by order_number
    .group_by("order_number")
    # aggregate by sum of item_total and count of line_item_id
    .agg([
        pl.col("item_total").sum().alias("calculated_total"),
        pl.count("line_item_id").alias("item_count")
    ])
)
detail_df.head()


order_number,calculated_total,item_count
str,f64,u32
"""PO-2023-001771""",365834.51,3
"""PO-2023-002322""",564059.57,3
"""PO-2023-000612""",578596.04,5
"""PO-2023-001533""",72771.7,2
"""PO-2023-003496""",499722.0,3


In [72]:
discrepancy_df = (
    # join detail and overview on order_number
    detail_df.join(order_summary_subset, how = 'inner', on = 'order_number')
    # create a total_discrepancy column
    .with_columns(
        (pl.col('total_amount') - pl.col('calculated_total')).alias('total_discrepancy')
    )
    # filter for orders where absolute value of total discrepancy > $0.01
    .filter(pl.col('total_discrepancy').abs() > 0.1)
    # select columns for final dataframe
    .select([
        'order_number', 'total_amount', 'calculated_total', 'total_discrepancy', 'vendor', 'destination_dc_code'
    ])
    .sort('order_number', descending = False)
)
discrepancy_df.head()

order_number,total_amount,calculated_total,total_discrepancy,vendor,destination_dc_code
str,f64,f64,f64,str,str
"""PO-2023-000003""",19697.01,474361.49,-454664.48,"""Pure Storage""","""DC-US006"""
"""PO-2023-000009""",648004.66,195241.08,452763.58,"""Super Micro Computer""","""DC-BR001"""
"""PO-2023-000031""",94887.07,32311.29,62575.78,"""Schneider Electric""","""DC-BR001"""
"""PO-2023-000041""",1.1879e6,128607.45,1.0593e6,"""Cisco Systems""","""DC-US002"""
"""PO-2023-000060""",4.0690e6,903581.64,3.1655e6,"""Lenovo""","""DC-SG002"""


The code below will calculate what fraction of 2023 orders has mismatched totals

In [73]:
count_orders_df = (
    # join detail and overview on order_number
    detail_df.join(order_summary_subset, how = 'inner', on = 'order_number')
    # calculate total number of unique order_numbers
    .select(
        pl.col('order_number').n_unique().alias('unique_order_count')
    )
)

print(count_orders_df)

shape: (1, 1)
┌────────────────────┐
│ unique_order_count │
│ ---                │
│ u32                │
╞════════════════════╡
│ 3430               │
└────────────────────┘


In [74]:
count_discrepancy_orders_df = (
    # use discrepancy_df created earlier
    discrepancy_df.select(pl.col('order_number').n_unique().alias('unique_order_count'))
)
print(count_discrepancy_orders_df)

shape: (1, 1)
┌────────────────────┐
│ unique_order_count │
│ ---                │
│ u32                │
╞════════════════════╡
│ 236                │
└────────────────────┘


In [75]:
num_discrepancy = count_discrepancy_orders_df.item()
num_total = count_orders_df.item()

# calculate percentage
percent_discrepancy = (num_discrepancy / num_total) * 100

print(f"Percent of orders with discrepancy: {percent_discrepancy:.2f}%")

Percent of orders with discrepancy: 6.88%


### Save Results

Write output.csv which gives the top 5 vendors per country by total_spend

In [76]:
df.write_csv('output.csv')

Write mismatches.csv which shows orders with total discrepancies > $0.01

In [77]:
discrepancy_df.write_csv('mismatches.csv')